In [1]:
# optional : install deps to make it work right
# %pip install -U bentoml fastapi starlette pydantic pydantic-core typing_extensions uvicorn scikit-learn
%pip install -r requirements.txt



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# define training data for our simple ai ops model

bad_texts = [
    "service is down in production",
    "database connection refused",
    "error 500 on checkout",
    "latency is spiking above 2s",
    "memory leak detected in worker",
    "disk is almost full on node 3",
    "cpu pegged at 100 percent",
    "timeout talking to upstream api",
    "critical alert: payment failures",
    "users cannot log in",
    "incident declared sev1",
    "pager going off nonstop",
    "major outage across regions",
    "kubernetes pod crashloopbackoff",
    "ssl certificate expired",
    "data corruption detected"
]

good_texts = [
    "deploy completed successfully",
    "all systems operational",
    "health check passing",
    "no issues detected",
    "latency back to normal",
    "service recovered after restart",
    "incident resolved",
    "error rate returned to baseline",
    "backup completed successfully",
    "scaling up completed",
    "cache warmed successfully",
    "migration finished without errors",
    "api responding normally",
    "monitoring is green",
    "rollout finished",
    "tests passed in ci"
]

train_texts = bad_texts + good_texts
train_labels = [1]*len(bad_texts) + [0]*len(good_texts)


In [3]:
# train the model
# %pip install -q scikit-learn

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

model = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1, 2))),
    ("clf", LogisticRegression(max_iter=1000))
])

model.fit(train_texts, train_labels)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidf', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


In [4]:
# optional, test the model inline

samples = [
    "checkout is failing with 500 errors",
    "deploy finished and everything looks good",
    "pods are crashlooping in prod",
    "all systems operational"
]

pred = model.predict(samples)
proba = model.predict_proba(samples)

for t, y, p in zip(samples, pred, proba):
    print(y, round(float(p[1]), 3), t)

1 0.541 checkout is failing with 500 errors
0 0.416 deploy finished and everything looks good
1 0.555 pods are crashlooping in prod
0 0.404 all systems operational


In [5]:
# save the model as bentoml package
import bentoml
model_ref = bentoml.sklearn.save_model(
    "ops_sentiment",
    model,
    signatures={"predict": {"batchable": True}, "predict_proba": {"batchable": True}},
)



In [ ]:
# optional run in uvicorn
#  !python -m uvicorn service:app --host 127.0.0.1 --port 7777 --reload

In [8]:
!python -m bentoml build
# optional list containers
# !python -m bentoml bentos list
# !python -m bentoml containerize ops_sentiment_service:latest


c:\Users\BAYONETTA V\AppData\Local\Programs\Python\Python311\Lib\site-packages\bentoml\io.py:7: BentoMLDeprecationWarning: `bentoml.io` is deprecated since BentoML v1.4 and will be removed in a future version. Please upgrade to new style IO types instead.
  warn_deprecated(
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\BAYONETTA V\AppData\Local\Programs\Python\Python311\Lib\site-packages\bentoml\__main__.py", line 4, in <module>
    cli()
  File "c:\Users\BAYONETTA V\AppData\Local\Programs\Python\Python311\Lib\site-packages\click\core.py", line 1485, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\BAYONETTA V\AppData\Local\Programs\Python\Python311\Lib\site-packages\click\core.py", line 1406, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "c:\Users\BAYONETTA V\AppData\Local\Programs\Python\Python311\L